### Querying via python in ibm-db


In [1]:
import ibm_db  #Importing ibm_db

In [2]:
#Database credentials specification

dsn_hostname = 'dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net'
dsn_uid = 'wqf33423'
dsn_pwd = '12ss77pz8z7d@3fj'
dsn_driver = '{IBM DB2 ODBC DRIVER}'
dsn_database = 'BLUDB'
dsn_port = '50000'
dsn_protocol = 'TCPIP'

In [3]:
# Specifying dsn string and Creating Database connection

dsn = (
    'DRIVER = {0};'
    'DATABASE = {1};'
    'HOSTNAME = {2};'
    'PORT = {3};'
    'PROTOCOL = {4};'
    'UID = {5};'
    'PWD = {6};').format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

print(dsn) #print to check values

try:
    conn = ibm_db.connect(dsn,"","")
    print('Connected to database: ',dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)
except:
    print('Unable to connect: ',ibm_db.conn_errormsg())


DRIVER = {IBM DB2 ODBC DRIVER};DATABASE = BLUDB;HOSTNAME = dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net;PORT = 50000;PROTOCOL = TCPIP;UID = wqf33423;PWD = 12ss77pz8z7d@3fj;
Connected to database:  BLUDB as user:  wqf33423 on host:  dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net


### Starting with Queries

In [24]:
#drop table query
dropQuery = 'DROP TABLE INSTRUCTOR' # Used to drop table if already exists
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

In [25]:
#create table query
createQuery = 'CREATE Table INSTRUCTOR(INS_ID INTEGER PRIMARY KEY NOT NULL,LASTNAME VARCHAR(30),FIRSTNAME VARCHAR(30),CITY VARCHAR(30),COUNTRY CHAR(2));'
createStmt = ibm_db.exec_immediate(conn, createQuery)

In [26]:
#insert into table query
insertQuery = "INSERT INTO INSTRUCTOR(INS_ID, LASTNAME, FIRSTNAME, CITY, COUNTRY) VALUES ('1', 'Ahuja', 'Rav', 'Toronto', 'CA'), ('2', 'Chong', 'Raul', 'Toronto', 'CA'), ('3', 'Vasudevan', 'Hima', 'Chicago','US');"
insertStatement = ibm_db.exec_immediate(conn, insertQuery)

In [34]:
#fetching query 1

selectQuery1 = 'SELECT * FROM INSTRUCTOR'
selectQuery1_Stmt = ibm_db.exec_immediate(conn, selectQuery1)
ibm_db.fetch_both(selectQuery1_Stmt) #COMMENT OUT TO SEE ALL ROWS IN THE NEXT CELL

{'INS_ID': 1,
 0: 1,
 'LASTNAME': 'Ahuja',
 1: 'Ahuja',
 'FIRSTNAME': 'Rav',
 2: 'Rav',
 'CITY': 'Toronto',
 3: 'Toronto',
 'COUNTRY': 'CA',
 4: 'CA'}

In [35]:
#fetching more rows

while ibm_db.fetch_row(selectQuery1_Stmt)!=False:
    print(" ID:",  ibm_db.result(selectQuery1_Stmt, 0), 
          " FNAME:",  ibm_db.result(selectQuery1_Stmt, "FIRSTNAME"), 
          " LNAME:",  ibm_db.result(selectQuery1_Stmt, "LASTNAME"), 
          " CITY:",  ibm_db.result(selectQuery1_Stmt, "CITY"), 
          " COUNTRY:",  ibm_db.result(selectQuery1_Stmt, "COUNTRY"))

 ID: 2  FNAME: Raul  LNAME: Chong  CITY: Toronto  COUNTRY: CA
 ID: 3  FNAME: Hima  LNAME: Vasudevan  CITY: Chicago  COUNTRY: US


In [39]:
# making a show all function

def showAll():
    S = "SELECT * FROM INSTRUCTOR"
    s = ibm_db.exec_immediate(conn, S)
    while ibm_db.fetch_row(s)!=False:
        print(" ID:",  ibm_db.result(s, 0), 
              " FNAME:",  ibm_db.result(s, "FIRSTNAME"), 
              " LNAME:",  ibm_db.result(s, "LASTNAME"), 
              " CITY:",  ibm_db.result(s, "CITY"), 
              " COUNTRY:",  ibm_db.result(s, "COUNTRY"))

In [40]:
# UPDATE DATA

updateQuery = "UPDATE INSTRUCTOR SET CITY = 'Moosetown' WHERE FIRSTNAME = 'Rav' AND LASTNAME = 'Ahuja';"
updateStmt = ibm_db.exec_immediate(conn, updateQuery)
showAll()

 ID: 1  FNAME: Rav  LNAME: Ahuja  CITY: Moosetown  COUNTRY: CA
 ID: 2  FNAME: Raul  LNAME: Chong  CITY: Toronto  COUNTRY: CA
 ID: 3  FNAME: Hima  LNAME: Vasudevan  CITY: Chicago  COUNTRY: US


In [43]:
# Using Pandas to retrieve data
import pandas
import ibm_db_dbi

In [44]:
#Connection formulation for pandas

pconn = ibm_db_dbi.Connection(conn)

In [46]:
# making dataframe from selectQuery1
df = pandas.read_sql(selectQuery1, pconn)
print(df)

   INS_ID   LASTNAME FIRSTNAME       CITY COUNTRY
0       1      Ahuja       Rav  Moosetown      CA
1       2      Chong      Raul    Toronto      CA
2       3  Vasudevan      Hima    Chicago      US


In [49]:
# we can use all pandas methods on this dataframe

print("The shape of dataframe is = ",df.shape)
print("Printing the LASTNAME column: ")
print(df.LASTNAME)

The shape of dataframe is =  (3, 5)
Printing the LASTNAME column: 
0        Ahuja
1        Chong
2    Vasudevan
Name: LASTNAME, dtype: object


In [50]:
# closing the connection

ibm_db.close(conn)

True